In [1]:
!pip install "ibm-watsonx-ai" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install "langchain==0.1.8" | tail -n 1
!pip install "langchain_ibm==0.0.1" | tail -n 1

In [2]:
import getpass

credentials = {
    "url": "https://jp-tok.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter): ········


In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [4]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}



In [5]:
from langchain_ibm import WatsonxLLM

granite_jp_llm = WatsonxLLM(
    model_id="ibm/granite-8b-japanese",
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
    params=parameters
    )

In [6]:
granite_jp_llm.dict()


{'model_id': 'ibm/granite-8b-japanese',
 'deployment_id': '',
 'params': {'decoding_method': 'sample',
  'max_new_tokens': 100,
  'min_new_tokens': 1,
  'temperature': 0.5,
  'top_k': 50,
  'top_p': 1},
 'project_id': '53afd616-fa11-4699-bd65-ea1e8ded9173',
 'space_id': '',
 '_type': 'IBM watsonx.ai'}

In [7]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"],
    template="""{topic}についてランダムな質問を生成してください。
                            質問: """
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="""次の質問に答えてください。: {question}
                            回答:""",
)

In [8]:
from langchain.chains import LLMChain

prompt_to_granite_jp = LLMChain(llm=granite_jp_llm, prompt=prompt_1, output_key='question')

In [9]:
granite_jp_2 = LLMChain(llm=granite_jp_llm, prompt=prompt_2, output_key='answer')

In [10]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_granite_jp, granite_jp_2], verbose=True)

In [11]:
qa.run("IBM")

/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
IBMは、コグニティブ・コンピューティング、クラウド・コンピューティング、モバイル・コンピューティング、ソーシャル・コンピューティング、ビッグ・データ分析、セキュリティ、およびモノのインターネット向けのオープン標準とテクノロジーの開発と促進を専門とするアメリカの多国籍テクノロジー企業です。IBMの最も有名な製品は何ですか？
 Watson

> Finished chain.


' Watson'